In [1]:
import arrow
import argparse            # Construct the argument parser
import logging
import numpy as np
import pandas as pd
import pyodbc
import time
import warnings

warnings.filterwarnings('ignore')

from datetime import datetime
from humanfriendly import format_timespan
from sqlalchemy import create_engine
from tqdm import tqdm

In [2]:
from osisoft.pidevclub.piwebapi.pi_web_api_client import PIWebApiClient
from osisoft.pidevclub.piwebapi.models import PIStreamValues, PITimedValue

from PiHelper import *

In [3]:
server = '192.168.5.191'
database = 'Runtime'
username = 'pi'
password = '1m4dm1n'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [50]:
piwebapi_url = 'https://192.168.5.74/piwebapi'
client = PIWebApiClient(piwebapi_url, useKerberos=False, username="administrator", password="Spc12345", verifySsl=False) 
piHelper = PiHelper(client)

In [60]:
# Read model configuration file
df_conf = pd.read_csv('AF_PTAR.csv')

# Filter model configuration based on object type
object_type = "ObjectType=='Attribute'"
df_conf = df_conf.query(object_type)
df_conf = df_conf.reset_index(drop=True)


In [30]:
# Create start date and end date table for datestamp selection
start = arrow.get("20220101 00:00:00")
end = arrow.get("20220315 23:59:59")
start_list = []
end_list = []

for r in arrow.Arrow.span_range('days', start, end):
    sdate = r[0].floor('day').format('YYYYMMDD HH:mm:ss')
    start_list.append(sdate)
    edate = r[1].floor('second').shift(days=+6).format('YYYYMMDD HH:mm:ss')
    end_list.append(edate)

df_date = pd.DataFrame(list(zip(start_list, end_list)), columns =['start_date', 'end_date'])
#df_date['start_date'] = pd.to_datetime(df_date['start_date'])
#df_date['end_date'] = pd.to_datetime(df_date['end_date'])

l = list(np.arange(0,len(df_date),7))
df_date = df_date.iloc[l].sort_values('start_date',ascending=False).reset_index()
df_date = df_date.drop(columns=['index'])

In [31]:
df_date

,start_date,end_date
0,20220312 00:00:00,20220318 23:59:59
1,20220305 00:00:00,20220311 23:59:59
2,20220226 00:00:00,20220304 23:59:59
3,20220219 00:00:00,20220225 23:59:59
4,20220212 00:00:00,20220218 23:59:59
5,20220205 00:00:00,20220211 23:59:59
6,20220129 00:00:00,20220204 23:59:59
7,20220122 00:00:00,20220128 23:59:59
8,20220115 00:00:00,20220121 23:59:59
9,20220108 00:00:00,20220114 23:59:59


In [62]:
# For test
tagname = df_conf['Name'][5]            # tag_path = '\\\\PISERVER\\Database1\\PCS0210PN001_Crushing|MAR.M0210_FE001_MI_sPV'
af_path = df_conf['AFPath'][5]

#logging.basicConfig(filename=tagname+'.log', filemode='a', level=logging.INFO, format='%(asctime)s - %(message)s', datefmt='%d-%b-%y %H:%M:%S')
tagname, af_path

('MAR.M0210_FE002_sHRSRUN',
 '\\\\PISERVER\\Database1\\PCS0210PN002_Reclaim|MAR.M0210_FE002_sHRSRUN')

In [33]:
startdate = df_date['start_date'][len(df_date)-1] #"20220330 00:00:00"
enddate = df_date['end_date'][len(df_date)-1] # "20220315 23:59:59"
startdate, enddate

('20220101 00:00:00', '20220107 23:59:59')

In [34]:
%%time
# Query to WW
sql = "SELECT DateTime, Value \
 FROM History \
 WHERE History.TagName = '{}' \
 AND wwRetrievalMode = 'Cyclic' \
 AND wwResolution = 1000 \
 AND wwQualityRule = 'Extended' \
 AND Quality = 0 \
 AND wwVersion = 'Latest' \
 AND DateTime >= '{}' \
 AND DateTime <= '{}' ".format(tagname,startdate,enddate)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 6.68 µs


In [35]:
t0 = time.time()
# Get data from WW Historian
df = pd.read_sql(sql, cnxn)
t1 = time.time()
execution_time = format_timespan(t1 - t0, True, max_units=4)
print(execution_time)

17 seconds, 658 milliseconds, 792 microseconds and 734.15 nanoseconds


In [36]:
len(df)

604800

In [37]:
%%time
# Get values
values = df['Value']

# Get timestamp
timestamps = df['DateTime']

CPU times: user 33 µs, sys: 0 ns, total: 33 µs
Wall time: 36.2 µs


In [ ]:
%%time
# Write data to PI tag
response = piHelper.insertTimeSeriesValues(af_path, values, timestamps)

In [ ]:
print(str(response))

In [ ]:
response

In [ ]:
if 202 in response:
    print("OK")

In [ ]:
# Read data
#tag_path = "pi:\\PISERVER\\" + tagname
attr_pi = "af:" + af_path

end = arrow.get(enddate,'YYYYMMDD HH:mm:ss').format('YYYY-MM-DD HH:mm:ss')
start = arrow.get(startdate,'YYYYMMDD HH:mm:ss').format('YYYY-MM-DD HH:mm:ss')

# Get data from PI Historian based on source_tag
#print("Get data from {}, {} ...".format(name, source_tag))
dp = piHelper.getInterpolatedData(path=attr_pi, start_time=start, end_time=end, interval='1M')

In [ ]:
dp.head(10)